# 에이플러스에셋을 위한 AI 자산배분 모델

## 프로젝트 구성

### <span style="color:yellow">1. 데이터 수집 및 가공</span>

- dataset_base_v2.py    : 데이터 수집을 위한 전반적인 모듈들
- dataset_v2.py         : 베이스 모듈 기반 커스텀 데이터셋 생성 (여기서 사용자데이터셋 정의)
- transforms_v2.py      : 데이터 가공 모듈

### 2. 모델 설계

- layer.py              : 기본적인 레이어 정의
- model_v2.py           : 레이어 기반으로 모델 작성 (여기서 사용자 모델 정의)

### 3. 학습

- main_v2.py            : 학습을 위한 모듈
    
### 4. 기타

- logger_v2.py          : 학습 과정 로깅




---


### 1. 데이터 수집


1.1. 데이터 로딩

- 다양한 데이터를 **더할 수 있게** 설계 (AddibleData: 기본적으로 left join 형식으로 합쳐짐)


In [1]:

# dataset_v2에 필요한 각각의 데이터 정의
from dataset_v2 import AplusData, MacroData

# 각 데이터 확보
aplus_data = AplusData()
macro_data = MacroData()

# 데이터 병합 
my_data = aplus_data + macro_data


print(my_data)

[head]:
              robo_agg       trend     income     krbond  usyc2y10 index  \
2001-01-03  100.000000  100.000000  100.00000  100.00000            37.1   
2001-01-04  100.042597  100.260692   99.54205  100.49596            47.0   

            usyc3m2y index  usdjpy curncy  usgg10yr index  nfp t index  \
2001-01-03          -58.05         113.58           5.158       132709   
2001-01-04          -54.70         115.55           5.039       132709   

            gdp cury index  gc1 comdty  cl1 comdty  hg1 comdty  spx index  \
2001-01-03             5.5       269.3       28.00        81.7    1347.56   
2001-01-04             5.5       268.4       28.14        81.1    1333.34   

            indu index  rty index  usdkrw curncy  copper_gold_r    spx_dj  \
2001-01-03    10945.75    484.430        1271.25       0.303379  0.123113   
2001-01-04    10912.41    477.234        1265.50       0.302161  0.122186   

              spx_rs  
2001-01-03  2.781743  
2001-01-04  2.793891  
[tail]:

1.2. 데이터 가공
- transforms_v2.py에서 데이터 가공을 위한 모듈 정의

- dataset_v2에서 transform_v2.py에 있는 모듈을 이용하여 _transform() 함수 정의
(dataset_base_v2.AddibleData.transform()에서 to_numpy -> _transform() -> to_df로 적용)

- 데이터 정의할 때, __init__에 label로 쓰려는 데이터 정의 (ex. dataset_v2.AplusData 참조)

- 참고:

    Transforms.apply() - 정의된 모듈을 병렬적/독립적으로 연산할 때 적용 (연산 후 늘어나는 columns 이름 필요)

    Transforms.sequential() - 정의된 모듈을 순차적으로 연산할 때 적용 (연산 후에도 사이즈 동일)

In [19]:
from collections import OrderedDict
from dataset_base_v2 import DataFromFiles


# 예시
class AplusData_example(DataFromFiles):
    def __init__(self, file_nm='app_data_20200630.txt'):
        super().__init__(file_nm)

        # label로 사용할 컬럼명 정의 (반드시 ordered_dict 형식, 순서 중요!)
        self.label_columns_dict = OrderedDict()
        for feature in ['logy', 'mu', 'sigma']:
            self.label_columns_dict[feature] = ['{}_{}'.format(col, feature) for col in self.columns]

    def _transform(self):
        # 여기에 dataframe에 적용할 모듈 추가
        transforms_apply = transforms_v2.Transforms([
            (transforms_v2.RollingLogReturn(20), 'logy'),
            (transforms_v2.RollingLogReturn(60), 'logy60'),
            (transforms_v2.RollingLogReturn(120), 'logy120'),
            (transforms_v2.RollingLogReturn(250), 'logy250'),
            (transforms_v2.RollingMeanReturn(250), 'mu'),
            (transforms_v2.RollingStdReturn(250), 'sigma'),
        ])

        # apply() 함수를 이용하여 병렬 적용 후 concatenate
        self.df, self.columns = transforms_apply.apply(
            self.df, self.columns,
            reduce='concat')

a = AplusData_example()
print(a)

[head]:
              robo_agg       trend     income     krbond
2001-01-03  100.000000  100.000000  100.00000  100.00000
2001-01-04  100.042597  100.260692   99.54205  100.49596
[tail]:
              robo_agg       trend      income      krbond
2020-06-29  379.432556  441.750866  284.206566  263.199062
2020-06-30  382.608476  444.323463  286.358456  262.915746
file dir: ['./data/app_data_20200630.txt']
shape: (4817, 4)



1.3. 데이터로더 생성

- AddibleData class를 이용하여 DatasetManager를 통해 torch의 data_loader 생성


In [14]:
from dataset_v2 import DatasetManager

# datamanager 정의
data_list = [AplusData(), MacroData()]
dm = DatasetManager(data_list, test_days=250, batch_size=256)

# t시점 (base_i) 기준 data_loader
train_loader = dm.get_data_loader(2000, 'train')

# 샘플 데이터
print(next(iter(train_loader)))


{'features_prev': tensor([[[-2.6087e-02, -5.7367e-02, -4.5521e-02,  ...,  1.2126e+00,
          -1.1954e+00,  0.0000e+00],
         [-3.9159e-02, -1.2798e-02, -1.2889e-03,  ...,  1.7423e+00,
          -9.9944e-01,  7.6923e-02],
         [ 6.0121e-03,  1.8616e-03,  4.1231e-03,  ...,  1.4607e+00,
          -1.1998e+00,  1.5385e-01],
         ...,
         [ 2.2866e-02, -2.3054e-03, -2.0807e-02,  ...,  1.3255e+00,
          -8.7115e-01,  7.6923e-01],
         [-1.0810e-02, -2.0589e-02, -3.8889e-02,  ...,  1.4745e+00,
          -9.8557e-01,  8.4615e-01],
         [-2.2455e-02, -6.4828e-03,  1.4189e-02,  ...,  2.1373e+00,
          -4.2419e-01,  9.2308e-01]],

        [[ 3.4086e-02,  3.0440e-02, -3.1782e-02,  ...,  4.1867e-01,
          -2.3125e+00,  0.0000e+00],
         [-4.8269e-02, -4.6338e-02,  5.6060e-03,  ...,  2.9170e-01,
          -1.1919e+00,  7.6923e-02],
         [-1.4389e-02, -9.1378e-03,  2.5561e-03,  ...,  1.7324e-02,
          -1.3788e+00,  1.5385e-01],
         ...,
       

---


### 2. Model 설계

2.1. layer

이것저것 필요한 커스텀모듈 정의


2.2. model_v2

![model_structure](./img/model_structure.png)


- model_v2.MyModel 에 구조 정의

- forward_with_loss를 통해 모델 학습


---

### 3. 학습

3.1. Configs class

- 학습을 위한 각종 파라미터 정의


3.2. Trainer class

- main_v2.py 에 위치

- 클래스 내에서 model 및 optimizer 정의

- train(), eval(), test() 함수 정의

- train&test를 통합적으로 실행하기 위한 run(), run_all() 함수 정의


3.3. main()

- Config파일 및 테스트이름 정의(Configs) -> 데이터 정의 (DatasetManager) -> 트레이너 정의(Trainer) -> Trainer.run()



In [ ]:
# !python main_v2.py